<a href="https://colab.research.google.com/github/kw908/RL_doorkick/blob/main/Suttonbookexercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 3.18 \\
Relationship between $V^\pi$ and $q^\pi$:
$$
V^\pi(s) = \sum_{a \in A} \pi(a|s)q^\pi(s,a)
$$

In [ ]:
## Exercise 3.14

import numpy as np

neighbors = [(0,1.0,2.3),
                   (0,1.0,0.4),
                   (0,1.0,-0.4),
                   (0,1.0,0.7)] # reward, probability, value(only accurate to one decimal place)
policy = np.array([0.25,0.25,0.25,0.25])
rewards = np.array([0,0,0,0])
middle_value = 0.7
gamma = 0.9
v_pi_middle = 0
state = [0,1]

for pa,v in zip(policy,neighbors):
       v_pi_middle += pa * v[1] * (v[0] + gamma*v[2])

print(v_pi_middle) #should evaluate to near 0.7


In [ ]:
## Example 4.1
%reload_ext autoreload
%autoreload 2

from Environments import gridworld_v0 as gw

THETA = 0.0001 #stop condition
GAMMA = 1.0 #non-discounted
episodes = 10 # number of episodes for policy evaluation
iteration = 1

env = gw.environment(sidelength=4,episodes=episodes) 

def policy_eval():
    k=0
    # for k in range(episodes):
    while(True):
        delta=0
        for s in env.states:
            v=s.value[k]
            for a in env.actions:
                next_state_value, reward = env.step(s,a,k)
                s.policy[a][2] = s.policy[a][1] * (reward + GAMMA*(next_state_value)) # calculate q
                s.value[k+1] += s.policy[a][0] * s.policy[a][2] # calculate v
            delta = max(delta, abs(v-s.value[k+1]))
        if delta<THETA or k==episodes: 
            print("Done evaluation~~~")
            return k
        k+=1

def policy_imp(k_terminal):
    for s in env.states:
        a_opt = []
        for a in env.actions:
            next_state_value, reward = env.step(s,a,k_terminal) # take one more step won't hurt I guess??
            s.policy[a][2] = s.policy[a][1] * (reward + GAMMA*(next_state_value))
        # update policy
        for a in env.actions:
            if s.policy[a][2] == max([i[2] for i in s.policy.values()]):
                a_opt.append(a) #find optimal action(s)
        for a in env.actions:
            if a in a_opt:
                s.policy[a][0] = 1/len(a_opt) #equally distribute probabilities among optimal actions
            else:
                s.policy[a][0] = 0
    print("Done update~~~")


def value_iter():
   k=0
   while(True):
        delta=0
        for s in env.states:
            v=s.value[k]
            for a in env.actions:
                next_state_value, reward = env.step(s,a,k)
                s.policy[a][2] = s.policy[a][1] * (reward + GAMMA*(next_state_value)) # calculate q
            s.value[k+1] = max([i[2] for i in s.policy.values()]) # update value with optimal policy in one sweep
            delta = max(delta, abs(v-s.value[k+1]))
        if delta<THETA or k==episodes: 
            print("Done value iteration~~~")
            break
        k+=1

        #update policy
        for s in env.states:
            for a in env.actions:
                next_state_value, reward = env.step(s,a,k_terminal) # take one more step won't hurt I guess??
                s.policy[a][2] = s.policy[a][1] * (reward + GAMMA*(next_state_value))
            # update policy
            for a in env.actions:
                if s.policy[a][2] == max([i[2] for i in s.policy.values()]):
                    a_opt = a
                    s.policy[a][0] = 1.0 #find optimal action(s)
                    break   
            for a in env.actions:
                if a is not a_opt:
                    s.policy[a][0] = 0 #equally distribute probabilities among optimal actions
        print("Done update~~~")
       


for i in range(iteration):
    k_terminal = policy_eval()
    policy_imp(k_terminal)

i = 0
for s in env.states:
    print(i, s.policy)
    i+=1

#visualize it~~
env.render()

       

